In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from scipy.stats import pearsonr
import shap
from sklearn.model_selection import train_test_split
from scipy.stats import pointbiserialr, shapiro, levene
from scipy.stats import mannwhitneyu
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
shap.initjs()
df = pd.read_csv(r'C:\Users\20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\output_combined_cleaned.csv')

In [ ]:
print(df.columns.tolist())

In [ ]:

df = pd.read_csv(r'C:\Users\20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\output_combined_cleaned.csv')
df= df[[  'MeanHR_3_prev_diff_3',
    'RMSSD_3_mean_diff_3',
 'MeanNN_3_mean_diff_3',
 'SDNN_3_mean_diff_3',
 'SDRMSSD_3_mean_diff_3',
 'LF/HF_3_mean_diff_3',
    'AU01_interaction_mean', 'AU01_interaction_std', 'AU02_interaction_mean', 'AU02_interaction_std', 'AU04_interaction_mean', 'AU04_interaction_std', 'AU05_interaction_mean', 'AU05_interaction_std', 'AU06_interaction_mean', 'AU06_interaction_std', 'AU07_interaction_mean', 'AU07_interaction_std', 'AU09_interaction_mean', 'AU09_interaction_std', 'AU10_interaction_mean', 'AU10_interaction_std', 'AU12_interaction_mean', 'AU12_interaction_std', 'AU14_interaction_mean', 'AU14_interaction_std', 'AU15_interaction_mean', 'AU15_interaction_std', 'AU17_interaction_mean', 'AU17_interaction_std', 'AU20_interaction_mean', 'AU20_interaction_std', 'AU23_interaction_mean', 'AU23_interaction_std', 'AU25_interaction_mean', 'AU25_interaction_std', 'AU26_interaction_mean', 'AU26_interaction_std', 'AU45_interaction_mean', 'AU45_interaction_std', "labels"]]
df_au = df[['AU01_interaction_mean', 'AU01_interaction_std', 'AU02_interaction_mean', 'AU02_interaction_std', 'AU04_interaction_mean', 'AU04_interaction_std', 'AU05_interaction_mean', 'AU05_interaction_std', 'AU06_interaction_mean', 'AU06_interaction_std', 'AU07_interaction_mean', 'AU07_interaction_std', 'AU09_interaction_mean', 'AU09_interaction_std', 'AU10_interaction_mean', 'AU10_interaction_std', 'AU12_interaction_mean', 'AU12_interaction_std', 'AU14_interaction_mean', 'AU14_interaction_std', 'AU15_interaction_mean', 'AU15_interaction_std', 'AU17_interaction_mean', 'AU17_interaction_std', 'AU20_interaction_mean', 'AU20_interaction_std', 'AU23_interaction_mean', 'AU23_interaction_std', 'AU25_interaction_mean', 'AU25_interaction_std', 'AU26_interaction_mean', 'AU26_interaction_std', 'AU45_interaction_mean', 'AU45_interaction_std', "labels"]]

df_hrv = df.drop(['AU01_interaction_mean', 'AU01_interaction_std', 'AU02_interaction_mean', 'AU02_interaction_std', 'AU04_interaction_mean', 'AU04_interaction_std', 'AU05_interaction_mean', 'AU05_interaction_std', 'AU06_interaction_mean', 'AU06_interaction_std', 'AU07_interaction_mean', 'AU07_interaction_std', 'AU09_interaction_mean', 'AU09_interaction_std', 'AU10_interaction_mean', 'AU10_interaction_std', 'AU12_interaction_mean', 'AU12_interaction_std', 'AU14_interaction_mean', 'AU14_interaction_std', 'AU15_interaction_mean', 'AU15_interaction_std', 'AU17_interaction_mean', 'AU17_interaction_std', 'AU20_interaction_mean', 'AU20_interaction_std', 'AU23_interaction_mean', 'AU23_interaction_std', 'AU25_interaction_mean', 'AU25_interaction_std', 'AU26_interaction_mean', 'AU26_interaction_std', 'AU45_interaction_mean', 'AU45_interaction_std'],axis=1)

In [ ]:

#force plot
df = pd.read_csv(r'C:\Users\20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\output_combined_cleaned.csv')
features = [
    'participant', 'MeanHR_3_prev_diff_3', 'RMSSD_3_mean_diff_3', 'MeanNN_3_mean_diff_3',
    'SDNN_3_mean_diff_3', 'SDRMSSD_3_mean_diff_3', 'LF/HF_3_mean_diff_3',
    'AU01_interaction_mean', 'AU01_interaction_std', 'AU02_interaction_mean', 'AU02_interaction_std',
    'AU04_interaction_mean', 'AU04_interaction_std', 'AU05_interaction_mean', 'AU05_interaction_std',
    'AU06_interaction_mean', 'AU06_interaction_std', 'AU07_interaction_mean', 'AU07_interaction_std',
    'AU09_interaction_mean', 'AU09_interaction_std', 'AU10_interaction_mean', 'AU10_interaction_std',
    'AU12_interaction_mean', 'AU12_interaction_std', 'AU14_interaction_mean', 'AU14_interaction_std',
    'AU15_interaction_mean', 'AU15_interaction_std', 'AU17_interaction_mean', 'AU17_interaction_std',
    'AU20_interaction_mean', 'AU20_interaction_std', 'AU23_interaction_mean', 'AU23_interaction_std',
    'AU25_interaction_mean', 'AU25_interaction_std', 'AU26_interaction_mean', 'AU26_interaction_std',
    'AU45_interaction_mean', 'AU45_interaction_std', 'labels'
]
df = df[features]

X = df.drop(columns=['labels'])
y = df['labels']

X_train, X_test, y_train, y_test = train_test_split(
    X.drop(columns=['participant']), y, test_size=0.2, random_state=42
)

model = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=200, subsample=0.8)
model.fit(X_train, y_train)

explainer = shap.Explainer(model)

# SHAP values for all data except participant
X_no_pid = X.drop(columns=['participant'])
shap_values = explainer(X_no_pid)

# Prepare DataFrame with shap values and participant ID
shap_df = pd.DataFrame(shap_values.values, columns=X_no_pid.columns)
shap_df['participant'] = X['participant'].values

# Group by participant
participant_shap = shap_df.groupby('participant').mean()
participant_features = X.groupby('participant').mean()

def plot_top_shap(pid, top_k=4):
    shap_vals = participant_shap.loc[pid].values
    feature_vals = participant_features.loc[pid].values
    feature_names = participant_features.columns

    top_indices = np.argsort(np.abs(shap_vals))[-top_k:]

    shap.force_plot(
        base_value=shap_values.base_values.mean(),
        shap_values=shap_vals[top_indices],
        features=feature_vals[top_indices],
        feature_names=feature_names[top_indices],
        matplotlib=True  # default
    )

plot_top_shap(pid=30, top_k=10)
plot_top_shap(pid=70, top_k=10)

In [ ]:
#split df data
x = df.drop('labels',axis=1)
y = df['labels']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

x_hrv = df_hrv.drop('labels',axis=1)
y_hrv = df_hrv['labels']
x_hrv_train, x_hrv_test, y_hrv_train, y_hrv_test = train_test_split(x_hrv, y_hrv, test_size=0.20, random_state=42)

x_au = df_au.drop('labels',axis=1)
y_au = df_au['labels']
x_au_train, x_au_test, y_au_train, y_au_test = train_test_split(x_au, y_au, test_size=0.20, random_state=42)

In [ ]:
# correlation matrix
cols = df.columns
n = len(cols)

corr_matrix = pd.DataFrame(np.zeros((n, n)), columns=cols, index=cols)
pval_matrix = pd.DataFrame(np.zeros((n, n)), columns=cols, index=cols)

for i in range(n):
    for j in range(n):
        if i <= j:
            corr, pval = pearsonr(df[cols[i]], df[cols[j]])
            corr_matrix.iloc[i, j] = corr
            corr_matrix.iloc[j, i] = corr
            pval_matrix.iloc[i, j] = pval
            pval_matrix.iloc[j, i] = pval

print("Correlation Matrix:\n", corr_matrix)
print("\nP-Value Matrix:\n", pval_matrix)


In [ ]:

if 'labels' not in df.columns:
    raise ValueError("Column 'labels' not found in DataFrame")
if not set(df['labels'].unique()).issubset({0, 1}):
    raise ValueError("Column 'labels' must be binary (0 and 1)")
results = []
for col in df.columns:
    if col != 'labels':
        group0 = df[df['labels'] == 0][col].dropna()
        group1 = df[df['labels'] == 1][col].dropna()
        corr, pval_corr = pointbiserialr(df[col], df['labels'])
        stat0, pval_shapiro0 = shapiro(group0) if len(group0) >= 3 else (None, None)
        stat1, pval_shapiro1 = shapiro(group1) if len(group1) >= 3 else (None, None)
        stat_levene, pval_levene = levene(group0, group1)

        results.append({
            'Variable': col,
            'Point-Biserial Corr': corr,
            'Correlation p-value': pval_corr,
            'Group0 Shapiro p-value': pval_shapiro0,
            'Group1 Shapiro p-value': pval_shapiro1,
            'Levene p-value': pval_levene,
            'Normality Assumption Met': (pval_shapiro0 is not None and pval_shapiro0 > 0.05) and
                                       (pval_shapiro1 is not None and pval_shapiro1 > 0.05),
            'Equal Variance Assumption Met': pval_levene > 0.05,
            'Significant Correlation (p<0.05)': pval_corr < 0.05
        })

results_df = pd.DataFrame(results)

print(results_df)

results_df.to_csv('pointbiserial_correlation_with_assumptions.csv', index=False)


In [ ]:
# rank biserial correlation
binary_cols = [col for col in df.columns if set(df[col].dropna().unique()).issubset({0, 1})]
other_cols = [col for col in df.columns if col not in binary_cols]
results = []
for bcol in binary_cols:
    for ocol in other_cols:
        group0 = df[df[bcol] == 0][ocol].dropna()
        group1 = df[df[bcol] == 1][ocol].dropna()

        if len(group0) > 0 and len(group1) > 0:
            u_stat, pval = mannwhitneyu(group0, group1, alternative='two-sided')
            r_rb = 1 - (2 * u_stat) / (len(group0) * len(group1))
            results.append({
                'Binary Label': bcol,
                'Variable': ocol,
                'Rank-Biserial Corr': r_rb,
                'Significant (p < 0.05)': 'Yes' if pval < 0.05 else 'No'
            })
results_df = pd.DataFrame(results)
results_df.to_csv('rankbiserial_results.csv', index=False)


In [ ]:
# Confusion matrix of best model, see below
confusion_matrix = np.array([[41, 28],
                             [28, 71]])
df_cm = pd.DataFrame(confusion_matrix, index=['Actual Negative (0)', 'Actual Positive (1)'],
                     columns=['Predicted Negative (0)', 'Predicted Positive (1)'])

print("Confusion Matrix:")
print(df_cm)

plt.figure(figsize=(6,4))
sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix: GradientBoostingClassifier")
plt.show()


In [ ]:
threshold = 0.8
high_corr_pairs = []

for i in range(n):
    for j in range(i+1, n):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            pair = (cols[i], cols[j], corr_matrix.iloc[i, j], pval_matrix.iloc[i, j])
            high_corr_pairs.append(pair)


print("Highly Correlated Pairs (|correlation| > 0.8):")
for col1, col2, corr, pval in high_corr_pairs:
    print(f"{col1} - {col2}: correlation = {corr:.3f}, p-value = {pval:.3e}")


In [ ]:
# FAU+HRV without class weights
specificity = make_scorer(recall_score, pos_label=0)

models = [
    SVC(C= 1000, gamma= 0.01, kernel = "rbf"),
    LogisticRegression(C=100,max_iter=1000, penalty= 'l2'),
    GradientBoostingClassifier(learning_rate= 0.1, max_depth= 5, n_estimators= 200, subsample=0.8),
    RandomForestClassifier(n_estimators=100, min_samples_split=2, max_depth= None),
]

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'recall0': specificity
}

for model in models:
    pipe = make_pipeline(StandardScaler(), model)

    scores = cross_validate(pipe, x_train, y_train, cv=5, scoring=scoring)
    print(f"\n{model.__class__.__name__}:")
    for metric in scoring:
        mean = np.mean(scores[f'test_{metric}']) * 100
        std = np.std(scores[f'test_{metric}']) * 100
        print(f"{metric.capitalize():<10} = {mean:.2f}% (+/- {std:.2f}%)")
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall (Sensitivity): {recall_score(y_test, y_pred) * 100:.2f}%")  # Class 1
    print(f"F1-Score: {f1_score(y_test, y_pred) * 100:.2f}%")
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall (Class 0): {recall_score(y_test, y_pred, pos_label=0) * 100:.2f}%")
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: {model.__class__.__name__}")
    plt.show()

In [ ]:
# FAU+HRV with class weights


specificity = make_scorer(recall_score, pos_label=0)

models = [
    SVC(class_weight = "balanced", C= 1000, gamma= 0.01, kernel = "rbf"),
    LogisticRegression(class_weight = "balanced", C=100,max_iter=1000, penalty= 'l2'),
    GradientBoostingClassifier(learning_rate= 0.1, max_depth= 5, n_estimators= 200, subsample=0.8),
    RandomForestClassifier(class_weight = "balanced", n_estimators=300, min_samples_split=10, max_depth= 20),
]

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'recall0': specificity
}

for model in models:
    pipe = make_pipeline(StandardScaler(), model)
    if isinstance(model, GradientBoostingClassifier):
        sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
        scores = cross_validate(pipe, x_train, y_train, cv=5, scoring=scoring, fit_params={'gradientboostingclassifier__sample_weight': sample_weights})
    else:
        scores = cross_validate(pipe, x_train, y_train, cv=5, scoring=scoring)
    print(f"\n{model.__class__.__name__}:")
    for metric in scoring:
        mean = np.mean(scores[f'test_{metric}']) * 100
        std = np.std(scores[f'test_{metric}']) * 100
        print(f"{metric.capitalize():<10} = {mean:.2f}% (+/- {std:.2f}%)")
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall (Sensitivity): {recall_score(y_test, y_pred) * 100:.2f}%")  # Class 1
    print(f"F1-Score: {f1_score(y_test, y_pred) * 100:.2f}%")
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall (Class 0): {recall_score(y_test, y_pred, pos_label=0) * 100:.2f}%")
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: {model.__class__.__name__}")
    plt.show()

In [ ]:
# shap
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
model = GradientBoostingClassifier(learning_rate= 0.1, max_depth= 5, n_estimators= 200, subsample=0.8
)
# rename HRV feature names e.g. RMSSD_3_mean_diff_3 to 'RMSSD' to make it more understandable
model.fit(x_train, y_train)
feature_names = [
'MeanHR_3_prev_diff_3', 'RMSSD', 'MeanNN', 'SDNN', 'SDRMSSD', 'LF/HF', 'AU01_interaction_mean', 'AU01_interaction_std', 'AU02_interaction_mean', 'AU02_interaction_std', 'AU04_interaction_mean', 'AU04_interaction_std', 'AU05_interaction_mean', 'AU05_interaction_std', 'AU06_interaction_mean', 'AU06_interaction_std', 'AU07_interaction_mean', 'AU07_interaction_std', 'AU09_interaction_mean', 'AU09_interaction_std', 'AU10_interaction_mean', 'AU10_interaction_std', 'AU12_interaction_mean', 'AU12_interaction_std', 'AU14_interaction_mean', 'AU14_interaction_std', 'AU15_interaction_mean', 'AU15_interaction_std', 'AU17_interaction_mean', 'AU17_interaction_std', 'AU20_interaction_mean', 'AU20_interaction_std', 'AU23_interaction_mean', 'AU23_interaction_std', 'AU25_interaction_mean', 'AU25_interaction_std', 'AU26_interaction_mean', 'AU26_interaction_std', 'AU45_interaction_mean', 'AU45_interaction_std'
]

x_test_renamed = x_test.copy()
x_test_renamed.columns = feature_names
explainer = shap.Explainer(model, x_test_renamed)
shap_values = explainer(x_test_renamed)
shap.summary_plot(shap_values, x_test_renamed)
shap.plots.bar(shap_values)


In [ ]:
# parameter tuning for HRV+AU for specificity


specificity = make_scorer(recall_score, pos_label=0)
param_grid = {
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [None,5, 10, 20],
    'randomforestclassifier__min_samples_split': [2, 5, 10]
}
model = RandomForestClassifier(class_weight="balanced")
pipe = make_pipeline(StandardScaler(), model)
search = RandomizedSearchCV(pipe, param_grid, scoring=specificity, cv=5)
search.fit(x_train, y_train)
print("Best parameters: randomforest", search.best_params_)
print("Best specificity score: randomforest", search.best_score_)

param_grid = {'svc__C': [0.1, 1, 10, 100, 1000],
			'svc__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
			'svc__kernel': ['rbf']}
model = SVC(class_weight="balanced")
pipe = make_pipeline(StandardScaler(), model)

grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)

grid.fit(x_train, y_train)
print("Best parameters: svc", grid.best_params_)
print("Best accuracy score: svc", grid.best_score_)




# Compute sample weights (like class_weight='balanced')
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

param_grid = {
    'gradientboostingclassifier__n_estimators': [100, 200],
    'gradientboostingclassifier__learning_rate': [0.01, 0.1],
    'gradientboostingclassifier__max_depth': [3, 5, 7],
    'gradientboostingclassifier__subsample': [0.8, 1.0]
}

model = GradientBoostingClassifier()
pipe = make_pipeline(StandardScaler(), model)

# Pass sample weights via fit_params
grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)
grid.fit(x_train, y_train, gradientboostingclassifier__sample_weight=sample_weights)

print("Best parameters: gradientboostingclassifier", grid.best_params_)
print("Best specificity score: gradientboostingclassifier", grid.best_score_)



param_grid = {'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [1, 10, 100, 1000]}
model = LogisticRegression(class_weight="balanced",max_iter=1000)
pipe = make_pipeline(StandardScaler(), model)
grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)
grid.fit(x_train, y_train)
print("Best parameters: logistic regression", grid.best_params_)
print("Best accuracy score: logistic regression", grid.best_score_)


In [ ]:
# parameter tuning for HRV+AU for accuracy


#random forest
specificity = make_scorer(recall_score, pos_label=0)
param_grid = {
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [None,5, 10, 20],
    'randomforestclassifier__min_samples_split': [2, 5, 10]
}
model = RandomForestClassifier(class_weight="balanced")
pipe = make_pipeline(StandardScaler(), model)
search = RandomizedSearchCV(pipe, param_grid, scoring="accuracy", cv=5)
search.fit(x_train, y_train)
print("Best parameters: randomforest", search.best_params_)
print("Best specificity score: randomforest", search.best_score_)


#SVC
param_grid = {'svc__C': [0.1, 1, 10, 100, 1000],
			'svc__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
			'svc__kernel': ['rbf']}
model = SVC(class_weight="balanced")
pipe = make_pipeline(StandardScaler(), model)

grid = GridSearchCV(pipe, param_grid, scoring="accuracy", cv=5)

grid.fit(x_train, y_train)
print("Best parameters: svc", grid.best_params_)
print("Best accuracy score: svc", grid.best_score_)


#gradientboostingclassifier
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

param_grid = {
    'gradientboostingclassifier__n_estimators': [100, 200],
    'gradientboostingclassifier__learning_rate': [0.01, 0.1],
    'gradientboostingclassifier__max_depth': [3, 5, 7],
    'gradientboostingclassifier__subsample': [0.8, 1.0]
}

model = GradientBoostingClassifier()
pipe = make_pipeline(StandardScaler(), model)

# Pass sample weights via fit_params
grid = GridSearchCV(pipe, param_grid, scoring="accuracy", cv=5)
grid.fit(x_train, y_train, gradientboostingclassifier__sample_weight=sample_weights)

print("Best parameters: gradientboostingclassifier", grid.best_params_)
print("Best specificity score: gradientboostingclassifier", grid.best_score_)


#logistic regression
param_grid = {'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [1, 10, 100, 1000]}
model = LogisticRegression(class_weight="balanced",max_iter=1000)
pipe = make_pipeline(StandardScaler(), model)
grid = GridSearchCV(pipe, param_grid, scoring="accuracy", cv=5)
grid.fit(x_train, y_train)
print("Best parameters: logistic regression", grid.best_params_)
print("Best accuracy score: logistic regression", grid.best_score_)


In [ ]:
# parameter tuning for HRV for specificity


specificity = make_scorer(recall_score, pos_label=0)
param_grid = {
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [None,5, 10, 20],
    'randomforestclassifier__min_samples_split': [2, 5, 10]
}
model = RandomForestClassifier(class_weight='balanced')
pipe = make_pipeline(StandardScaler(), model)
search = RandomizedSearchCV(pipe, param_grid, scoring=specificity, cv=5)
search.fit(x_hrv_train, y_hrv_train)
print("Best parameters: randomforest", search.best_params_)
print("Best specificity score: randomforest", search.best_score_)

param_grid = {'svc__C': [0.1, 1, 10, 100, 1000],
			'svc__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
			'svc__kernel': ['rbf']}
model = SVC(class_weight='balanced')  # Auto adjust weights inversely proportional to class frequencies
pipe = make_pipeline(StandardScaler(), model)

grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)

grid.fit(x_hrv_train, y_hrv_train)
print("Best parameters: svc", grid.best_params_)
print("Best accuracy score: svc", grid.best_score_)


sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
param_grid = {
    'gradientboostingclassifier__n_estimators': [100, 200],
    'gradientboostingclassifier__learning_rate': [0.01, 0.1],
    'gradientboostingclassifier__max_depth': [3, 5, 7],
    'gradientboostingclassifier__subsample': [0.8, 1.0]
}
model = GradientBoostingClassifier()
pipe = make_pipeline(StandardScaler(), model)
grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)
grid.fit(x_hrv_train, y_hrv_train, gradientboostingclassifier__sample_weight=sample_weights)
print("Best parameters: gradientboostingclassifier", grid.best_params_)
print("Best accuracy score: gradientboostingclassifier", grid.best_score_)

param_grid = {'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [1, 10, 100, 1000]}
model = LogisticRegression(class_weight='balanced', max_iter=1000)
pipe = make_pipeline(StandardScaler(), model)
grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)
grid.fit(x_hrv_train, y_hrv_train)
print("Best parameters: logistic regression", grid.best_params_)
print("Best accuracy score: logistic regression", grid.best_score_)


In [ ]:
# parameter tuning for AU for specificity


specificity = make_scorer(recall_score, pos_label=0)
param_grid = {
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [None,5, 10, 20],
    'randomforestclassifier__min_samples_split': [2, 5, 10]
}
model = RandomForestClassifier(class_weight='balanced')
pipe = make_pipeline(StandardScaler(), model)
search = RandomizedSearchCV(pipe, param_grid, scoring=specificity, cv=5)
search.fit(x_au_train, y_au_train)
print("Best parameters: randomforest", search.best_params_)
print("Best specificity score: randomforest", search.best_score_)

param_grid = {'svc__C': [0.1, 1, 10, 100, 1000],
			'svc__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
			'svc__kernel': ['rbf']}
model = SVC(class_weight='balanced')
pipe = make_pipeline(StandardScaler(), model)

grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)

grid.fit(x_au_train, y_au_train)
print("Best parameters: svc", grid.best_params_)
print("Best accuracy score: svc", grid.best_score_)

sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
param_grid = {
    'gradientboostingclassifier__n_estimators': [100, 200],
    'gradientboostingclassifier__learning_rate': [0.01, 0.1],
    'gradientboostingclassifier__max_depth': [3, 5, 7],
    'gradientboostingclassifier__subsample': [0.8, 1.0]
}
model = GradientBoostingClassifier()
pipe = make_pipeline(StandardScaler(), model)
grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)
grid.fit(x_au_train, y_au_train, gradientboostingclassifier__sample_weight=sample_weights )
print("Best parameters: gradientboostingclassifier", grid.best_params_)
print("Best accuracy score: gradientboostingclassifier", grid.best_score_)

param_grid = {'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [1, 10, 100, 1000]}
model = LogisticRegression(class_weight='balanced',max_iter=1000)
pipe = make_pipeline(StandardScaler(), model)
grid = GridSearchCV(pipe, param_grid, scoring=specificity, cv=5)
grid.fit(x_au_train, y_au_train)
print("Best parameters: logistic regression", grid.best_params_)
print("Best accuracy score: logistic regression", grid.best_score_)


In [ ]:
#HRV

specificity = make_scorer(recall_score, pos_label=0)

models = [
    SVC(C=1000, gamma=0.1, kernel='rbf', class_weight='balanced'),
    LogisticRegression(C=1, penalty="l2",max_iter=1000, class_weight='balanced'),
    GradientBoostingClassifier(learning_rate=0.1, max_depth=7, n_estimators=200, subsample=0.8),  # no class_weight param
    RandomForestClassifier(n_estimators=300, min_samples_split=2, max_depth=20, class_weight='balanced'),
]

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'recall0': specificity
}

for model in models:
    pipe = make_pipeline(StandardScaler(), model)

    if isinstance(model, GradientBoostingClassifier):
        sample_weights = compute_sample_weight(class_weight='balanced', y=y_hrv_train)
        pipe.fit(x_hrv_train, y_hrv_train, gradientboostingclassifier__sample_weight=sample_weights)

        y_hrv_pred = pipe.predict(x_hrv_test)

        print(f"\n{model.__class__.__name__} (with sample_weight):")
        print(f"Accuracy: {accuracy_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"Precision: {precision_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"Recall (Sensitivity): {recall_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"F1-Score: {f1_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"ROC AUC: {roc_auc_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"Recall (Class 0): {recall_score(y_hrv_test, y_hrv_pred, pos_label=0) * 100:.2f}%")
        print("Confusion Matrix:")
        print(confusion_matrix(y_hrv_test, y_hrv_pred))
    else:
        scores = cross_validate(pipe, x_hrv_train, y_hrv_train, cv=5, scoring=scoring)
        print(f"\n{model.__class__.__name__}:")
        for metric in scoring:
            mean = np.mean(scores[f'test_{metric}']) * 100
            std = np.std(scores[f'test_{metric}']) * 100
            print(f"{metric.capitalize():<10} = {mean:.2f}% (+/- {std:.2f}%)")
        pipe.fit(x_hrv_train, y_hrv_train)
        y_hrv_pred = pipe.predict(x_hrv_test)
        print(f"Accuracy: {accuracy_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"Precision: {precision_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"Recall (Sensitivity): {recall_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"F1-Score: {f1_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"ROC AUC: {roc_auc_score(y_hrv_test, y_hrv_pred) * 100:.2f}%")
        print(f"Recall (Class 0): {recall_score(y_hrv_test, y_hrv_pred, pos_label=0) * 100:.2f}%")
        print("Confusion Matrix:")
        print(confusion_matrix(y_hrv_test, y_hrv_pred))


In [ ]:
# AU

specificity = make_scorer(recall_score, pos_label=0)

models = [
    SVC(class_weight = "balanced",C=100, gamma= 0.01, kernel= 'rbf'),
    LogisticRegression(class_weight = "balanced", C=100,max_iter=1000, penalty ='l2'),
    GradientBoostingClassifier(learning_rate= 0.01, max_depth= 3, n_estimators= 200, subsample=0.8),
    RandomForestClassifier(class_weight = "balanced", n_estimators=100, min_samples_split=10, max_depth= 5),

]

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'recall0': specificity
}

for model in models:
    pipe = make_pipeline(StandardScaler(), model)
    if isinstance(model, GradientBoostingClassifier):
        sample_weights = compute_sample_weight(class_weight='balanced', y=y_au_train)
        scores = cross_validate(pipe, x_au_train, y_au_train, cv=5, scoring=scoring, fit_params={'gradientboostingclassifier__sample_weight': sample_weights})
    else:
        scores = cross_validate(pipe, x_au_train, y_au_train, cv=5, scoring=scoring)
    print(f"\n{model.__class__.__name__}:")
    for metric in scoring:
        mean = np.mean(scores[f'test_{metric}']) * 100
        std = np.std(scores[f'test_{metric}']) * 100
        print(f"{metric.capitalize():<10} = {mean:.2f}% (+/- {std:.2f}%)")
    pipe.fit(x_au_train, y_au_train)
    y_au_pred = pipe.predict(x_au_test)
    print(f"Accuracy: {accuracy_score(y_au_test, y_au_pred) * 100:.2f}%")
    print(f"Precision: {precision_score(y_au_test, y_au_pred) * 100:.2f}%")
    print(f"Recall (Sensitivity): {recall_score(y_au_test, y_au_pred) * 100:.2f}%")  # Class 1
    print(f"F1-Score: {f1_score(y_au_test, y_au_pred) * 100:.2f}%")
    print(f"ROC AUC: {roc_auc_score(y_au_test, y_au_pred) * 100:.2f}%")
    print(f"Recall (Class 0): {recall_score(y_au_test, y_au_pred, pos_label=0) * 100:.2f}%")
    cm = confusion_matrix(y_au_test, y_au_pred)
    print("Confusion Matrix:")
    print(cm)

In [ ]:
#search for interactions, SHAP dependency plots

sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# Train model
model = GradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=5,
    n_estimators=200,
    subsample=0.8
)
model.fit(x_train, y_train, sample_weight=sample_weights)
model.fit(x_train, y_train)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_test)
shap.dependence_plot("MeanNN_3_mean_diff_3", shap_values, x_test)